In [1]:
#!python3.8 -m pip install fastcore

In [2]:
import sys
import time
import odrive
from odrive.enums import *
import fibre.libfibre
# from fastcore.nb_imports import *
from fastcore.all import *

In [3]:
PERFORM_TESTS = True

In [4]:
class OdriveMotorController:
    """
    Class for configuring an Odrive axis for a Hoverboard motor. 
    Only works with one Odrive at a time.
    """
    FIND_ODRIVE_RETRIES = 10 # Number of retries to find odrive after reset with 1 sec delay per try
    MOTOR_KV = 10.0 # Motor KV
    # Min/Max phase inductance of motor
    MIN_PHASE_INDUCTANCE = 0.0008 #Original: 0 : 0.0017 for up to 6va : 0.0012 for 7
    MAX_PHASE_INDUCTANCE = 0.0024 #Measured: 0.0019619388040155172   Original: 0.001
    # Min/Max phase resistance of motor
    MIN_PHASE_RESISTANCE = 0.77 #Original: 0.0
    MAX_PHASE_RESISTANCE = 0.92 #Measured: 0.8209025263786316 Original 0.5
    # Tolerance for encoder offset float
    ENCODER_IS_HALL = False
    ENCODER_OFFSET_FLOAT_TOLERANCE = 0.05 #Only relevant for hall sensors I think?
    
    def __init__(
        self, 
        selected_axis_num:int,
        find_odrive_max_retries:int=10
    ):
        """
        Initalizes HBMotorConfig class by finding odrive, erase its 
        configuration, and grabbing specified axis object.
        
        :param axis_num: Which channel/motor on the odrive your referring to.
        :type axis_num: int (0 or 1)
        """
        self.selected_axis_num = selected_axis_num
        self.selected_axis = None
        self.find_odrive_max_retries = find_odrive_max_retries
        
        self._find_odrive()
        
    def _find_odrive(self):
        """
        Connect to Odrive
        """
        odrive_found = False
        for i in range(self.find_odrive_max_retries):
            try:
                self.odrv = odrive.find_any()
                odrive_found = True
                break
            except fibre.libfibre.ObjectLostError:
                print('Odrive temporarily not found. Trying again....')
                time.sleep(.5)
        if odrive_found == False: raise Exception("Odrive Not Found")
        self.selected_axis = getattr(self.odrv, "axis{}".format(self.selected_axis_num))

In [5]:
@patch
def dump_errors(self:OdriveMotorController):
    dump_errors(self.odrv)

In [6]:
if PERFORM_TESTS: omc = OdriveMotorController(selected_axis_num=1)

In [7]:
@patch
def select_axis(self:OdriveMotorController, selected_axis_num:int=0):
    self.selected_axis_num = selected_axis_num
    self._find_odrive()

In [8]:
if PERFORM_TESTS:
    temp_orig_selected_axis_num = omc.selected_axis_num
    omc.select_axis(selected_axis_num=0)
    assert omc.selected_axis_num == 0
    assert omc.selected_axis == omc.odrv.axis0
    omc.select_axis(selected_axis_num=1)
    assert omc.selected_axis_num == 1
    assert omc.selected_axis == omc.odrv.axis1
    omc.select_axis(selected_axis_num=temp_orig_selected_axis_num)

In [9]:
@patch
def _wait_idle(self:OdriveMotorController):
    """
    Waits until axis is idle
    """
    time.sleep(.1)
    while self.selected_axis.current_state != AXIS_STATE_IDLE:
        time.sleep(.1)

In [10]:
@patch
def _save_config(self:OdriveMotorController):
    """
    Save the Odrive configuration and reboot
    """
    try:
        self.odrv.save_configuration()
    except fibre.libfibre.ObjectLostError:
        pass
    print("Configuration Saved.")
    self._find_odrive()
    self._wait_idle()

In [11]:
if PERFORM_TESTS: omc._save_config()

05:08:53.521180214 [LEGACY_OBJ] protocol failed with 3 - propagating error to application


Configuration Saved.


In [12]:
@patch
def _erase_odrive(self:OdriveMotorController):
    print("Erasing Odrive configuration...")
    try:
        self.odrv.erase_configuration()
    except fibre.libfibre.ObjectLostError:
        pass
    self._find_odrive()
    self._wait_idle()

In [13]:
#if PERFORM_TESTS: omc._erase_odrive()

In [14]:
@patch
def _reboot_odrive(self:OdriveMotorController):
    print('Rebooting Odrive...')
    try:
        self.odrv.reboot()
    except fibre.libfibre.ObjectLostError:
        pass
    self._find_odrive()
    self._wait_idle()

In [15]:
if PERFORM_TESTS: omc._reboot_odrive()

Rebooting Odrive...


05:09:03.973686485 [LEGACY_OBJ] protocol failed with 3 - propagating error to application


In [16]:
@patch
def _config_brake_resistor(self:OdriveMotorController,
                           enable_brake_resistor:bool=True,
                           brake_resistance:float = 2.,
                           save_config=False):
    """
    Configure the brake resistor
    """
    self.odrv.config.enable_brake_resistor = True
    self.odrv.config.brake_resistance = 2.
    if save_config: self._save_config()

In [17]:
if PERFORM_TESTS: omc._config_brake_resistor()

In [18]:
@patch
def _configure_motor(self:OdriveMotorController):
    """
    
    I_bus_hard_max: inf (float)
    I_bus_hard_min: -inf (float)
    I_leak_max: 0.10000000149011612 (float)
    R_wL_FF_enable: False (bool)
    acim_autoflux_attack_gain: 10.0 (float)
    acim_autoflux_decay_gain: 1.0 (float)
    acim_autoflux_enable: False (bool)
    acim_autoflux_min_Id: 10.0 (float)
    acim_gain_min_flux: 10.0 (float)
    bEMF_FF_enable: False (bool)
x    calibration_current: 10.0 (float)
x    current_control_bandwidth: 1000.0 (float)
x    current_lim: 10.0 (float)
    current_lim_margin: 8.0 (float)
    dc_calib_tau: 0.20000000298023224 (float)
    inverter_temp_limit_lower: 100.0 (float)
    inverter_temp_limit_upper: 120.0 (float)
    motor_type: 0 (uint8)
    phase_inductance: 0.0 (float)
    phase_resistance: 0.0 (float)
x    pole_pairs: 7 (int32)
.    pre_calibrated: False (bool)
x    requested_current_range: 60.0 (float)
x    resistance_calib_max_voltage: 2.0 (float)
x    torque_constant: 0.03999999910593033 (float)
    torque_lim: inf (float)
    """
    
    # Hoverboard hub motors are quite high resistance compared to the hobby 
    # aircraft motors, so we want to use a bit higher voltage for the motor 
    # calibration, and set up the current sense gain to be more sensitive. 
    # The motors are also fairly high inductance, so we need to reduce the 
    # bandwidth of the current controller from the default to keep it 
    # stable.
    self.selected_axis.motor.config.resistance_calib_max_voltage = 12 #Original: 4
    self.selected_axis.motor.config.requested_current_range      = 24 #Original: 25
    self.selected_axis.motor.config.current_control_bandwidth    = 150 #Original: 100
    self.selected_axis.motor.config.current_lim = 14. #New was 8
    self.selected_axis.motor.config.calibration_current = 4. #New Original: 3
    self.selected_axis.motor.config.motor_type = MOTOR_TYPE_GIMBAL #New
    
    # Standard 6.5 inch hoverboard hub motors have 30 permanent magnet 
    # poles, and thus 15 pole pairs
    self.selected_axis.motor.config.pole_pairs = 15
    
    # Estimated KV but should be measured using the "drill test", which can
    # be found here:
    # https://discourse.odriverobotics.com/t/project-hoverarm/441
    self.selected_axis.motor.config.torque_constant = 8.27 / self.MOTOR_KV
    
    self._save_config()

    #input("Make sure the motor is free to move, then press enter...") #TODO: Re-enable

    print("Calibrating Odrive for hoverboard motor (you should hear a "
    "beep)...")

    self.selected_axis.requested_state = AXIS_STATE_MOTOR_CALIBRATION
    while self.selected_axis.current_state == AXIS_STATE_UNDEFINED: time.sleep(.1)
    while self.selected_axis.current_state == AXIS_STATE_MOTOR_CALIBRATION: time.sleep(.1)
    
    
    if self.selected_axis.motor.error != 0:
        print("Error: Odrive reported an error of {} while in the state " 
        "AXIS_STATE_MOTOR_CALIBRATION. Printing out Odrive motor data for "
        "debug:\n{}".format(self.selected_axis.motor.error, 
                            self.selected_axis.motor))

        raise Exception("Motor Parameter Error")

    if self.selected_axis.motor.config.motor_type != MOTOR_TYPE_GIMBAL:
        if self.selected_axis.motor.config.phase_inductance <= self.MIN_PHASE_INDUCTANCE or \
        self.selected_axis.motor.config.phase_inductance >= self.MAX_PHASE_INDUCTANCE:
            print("Error: After odrive motor calibration, the phase inductance "
            "is at {}, which is outside of the expected range. Either widen the "
            "boundaries of MIN_PHASE_INDUCTANCE and MAX_PHASE_INDUCTANCE (which "
            "is between {} and {} respectively) or debug/fix your setup. Printing "
            "out Odrive motor data for debug:\n{}".format(self.selected_axis.motor.config.phase_inductance, 
                                                          self.MIN_PHASE_INDUCTANCE,
                                                          self.MAX_PHASE_INDUCTANCE, 
                                                          self.selected_axis.motor))

            raise Exception("Motor Parameter Error")

        if self.selected_axis.motor.config.phase_resistance <= self.MIN_PHASE_RESISTANCE or \
        self.selected_axis.motor.config.phase_resistance >= self.MAX_PHASE_RESISTANCE:
            print("Error: After odrive motor calibration, the phase resistance "
            "is at {}, which is outside of the expected range. Either raise the "
            "MAX_PHASE_RESISTANCE (which is between {} and {} respectively) or "
            "debug/fix your setup. Printing out Odrive motor data for " 
            "debug:\n{}".format(self.selected_axis.motor.config.phase_resistance, 
                                self.MIN_PHASE_RESISTANCE,
                                self.MAX_PHASE_RESISTANCE, 
                                self.selected_axis.motor))

            raise Exception("Motor Parameter Error")

    # If all looks good, then lets tell ODrive that saving this calibration 
    # to persistent memory is OK
    self.selected_axis.motor.config.pre_calibrated = True
    self._save_config()
    

In [19]:
if PERFORM_TESTS: 
    omc._configure_motor()
    print(omc.selected_axis.motor.config.phase_inductance,omc.selected_axis.motor.config.phase_resistance)
    print(omc.selected_axis.motor.config.resistance_calib_max_voltage)
    print(omc.selected_axis.motor.is_calibrated)
    print(omc.selected_axis.motor.is_armed)
    
    

05:09:16.521386376 [LEGACY_OBJ] protocol failed with 3 - propagating error to application


Configuration Saved.
Calibrating Odrive for hoverboard motor (you should hear a beep)...
Configuration Saved.
0.0 0.0
12.0
True
False


In [20]:
omc.selected_axis.motor.error

0

In [21]:
# omc.dump_errors()
# dump_errors(omc.odrv)
# dump_errors(omc.selected_axis.motor.error)
# omc.dump_errors()
# dump_errors(omc.odrv)


In [22]:
@patch
def _configure_encoder(self:OdriveMotorController):
    """
    
    abs_spi_cs_gpio_pin: 1 (uint16)
x    bandwidth: 1000.0 (float)
    calib_range: 0.019999999552965164 (float)
xx    calib_scan_distance: 50.26548385620117 (float)
    calib_scan_omega: 12.566370964050293 (float)
x    cpr: 8192 (int32)
    direction: 0 (int32)
    enable_phase_interpolation: True (bool)
    find_idx_on_lockin_only: False (bool)
    hall_polarity: 0 (uint8)
    hall_polarity_calibrated: False (bool)
    ignore_illegal_hall_state: False (bool)
    index_offset: 0.0 (float)
x    mode: 0 (uint16)
    phase_offset: 0 (int32)
    phase_offset_float: 0.0 (float)
    pre_calibrated: False (bool)
    sincos_gpio_pin_cos: 4 (uint16)
    sincos_gpio_pin_sin: 3 (uint16)
    use_index: False (bool)
    use_index_offset: True (bool)
    """
    # Hoverboard motors contain hall effect sensors instead of incremental 
    # encorders
    self.selected_axis.encoder.config.mode = ENCODER_MODE_INCREMENTAL #Original: ENCODER_MODE_HALL

    # The hall feedback has 6 states for every pole pair in the motor. Since
    # we have 15 pole pairs, we set the cpr to 15*6 = 90.
    self.selected_axis.encoder.config.cpr = 3200 #Original: 90

    # Since hall sensors are low resolution feedback, we also bump up the 
    #offset calibration displacement to get better calibration accuracy.
    self.selected_axis.encoder.config.calib_scan_distance = 50 #Original: 150
    
    # Since the hall feedback only has 90 counts per revolution, we want to 
    # reduce the velocity tracking bandwidth to get smoother velocity 
    # estimates. We can also set these fairly modest gains that will be a
    # bit sloppy but shouldn’t shake your rig apart if it’s built poorly. 
    # Make sure to tune the gains up when you have everything else working 
    # to a stiffness that is applicable to your application.
    self.selected_axis.encoder.config.bandwidth = 200 #Original: 100 Last: 200
    
    print("Calibrating Odrive for encoder...")
    self.selected_axis.requested_state = AXIS_STATE_ENCODER_OFFSET_CALIBRATION
    while self.selected_axis.current_state != AXIS_STATE_ENCODER_OFFSET_CALIBRATION: time.sleep(.1)
    while self.selected_axis.current_state == AXIS_STATE_ENCODER_OFFSET_CALIBRATION: time.sleep(.1)

    # Wait for calibration to take place
    # time.sleep(20)

    if self.selected_axis.encoder.error != 0:
        print("Error: Odrive reported an error of {} while in the state "
        "AXIS_STATE_ENCODER_OFFSET_CALIBRATION. Printing out Odrive encoder "
        "data for debug:\n{}".format(self.selected_axis.encoder.error, 
                                     self.selected_axis.encoder))

        raise Exception("Encoder Parameter Error")

    if self.ENCODER_IS_HALL:
        # If offset_float isn't 0.5 within some tolerance, or its not 1.5 within
        # some tolerance, raise an error
        if not ((self.selected_axis.encoder.config.offset_float > 0.5 - self.ENCODER_OFFSET_FLOAT_TOLERANCE and \
        self.selected_axis.encoder.config.offset_float < 0.5 + self.ENCODER_OFFSET_FLOAT_TOLERANCE) or \
        (self.selected_axis.encoder.config.offset_float > 1.5 - self.ENCODER_OFFSET_FLOAT_TOLERANCE and \
        self.selected_axis.encoder.config.offset_float < 1.5 + self.ENCODER_OFFSET_FLOAT_TOLERANCE)):
            print("Error: After odrive encoder calibration, the 'offset_float' "
            "is at {}, which is outside of the expected range. 'offset_float' "
            "should be close to 0.5 or 1.5 with a tolerance of {}. Either "
            "increase the tolerance or debug/fix your setup. Printing out "
            "Odrive encoder data for debug:\n{}".format(self.selected_axis.encoder.config.offset_float, 
                                                        self.ENCODER_OFFSET_FLOAT_TOLERANCE, 
                                                        self.selected_axis.encoder))

            raise Exception("Encoder Parameter Error")
    else:
        if (self.selected_axis.encoder.config.phase_offset_float > -0.0001) and (self.selected_axis.encoder.config.phase_offset_float < 0.0001):
            print("Error: After odrive encoder calibration with non-hall sensor, the 'phase_offset_float' "
            "is at {}, which is outside of the expected range. 'phase_offset_float' "
            "should be not be 0.0 a tolerance of {}. Either "
            "increase the tolerance or debug/fix your setup. Printing out "
            "Odrive encoder data for debug:\n{}".format(self.selected_axis.encoder.config.phase_offset_float, 
                                                        self.ENCODER_OFFSET_FLOAT_TOLERANCE, 
                                                        self.selected_axis.encoder))
            raise Exception("Encoder Parameter Error")

    # If all looks good, then lets tell ODrive that saving this calibration 
    # to persistent memory is OK
    # self.odrv_axis.encoder.config.pre_calibrated = True #TODO: Not sure if we can save calibration for encoder
    self.selected_axis.config.startup_encoder_offset_calibration = True
    # self.odrv_axis.config.startup_closed_loop_control = True

    self.selected_axis.encoder.config.calib_scan_distance = 10 #Set shorter encoder calibration after startup after config complete

    print("Saving calibration configuration and rebooting...")
    self._save_config()
    
    
    

In [23]:
if PERFORM_TESTS: omc._configure_encoder();omc._reboot_odrive()

Calibrating Odrive for encoder...
Saving calibration configuration and rebooting...


05:09:44.1597548 [LEGACY_OBJ] protocol failed with 3 - propagating error to application


Configuration Saved.
Rebooting Odrive...


05:09:48.958370455 [LEGACY_OBJ] protocol failed with 3 - propagating error to application


In [24]:
omc.selected_axis.encoder.error

0

In [25]:
omc.selected_axis.encoder.config

abs_spi_cs_gpio_pin: 1 (uint16)
bandwidth: 200.0 (float)
calib_range: 0.019999999552965164 (float)
calib_scan_distance: 10.0 (float)
calib_scan_omega: 12.566370964050293 (float)
cpr: 3200 (int32)
direction: -1 (int32)
enable_phase_interpolation: True (bool)
find_idx_on_lockin_only: False (bool)
hall_polarity: 0 (uint8)
hall_polarity_calibrated: False (bool)
ignore_illegal_hall_state: False (bool)
index_offset: 0.0 (float)
mode: 0 (uint16)
phase_offset: -167 (int32)
phase_offset_float: 0.16792216897010803 (float)
pre_calibrated: False (bool)
sincos_gpio_pin_cos: 4 (uint16)
sincos_gpio_pin_sin: 3 (uint16)
use_index: False (bool)
use_index_offset: True (bool)

In [26]:
@patch
def _configure_controller(self:OdriveMotorController):
    self.selected_axis.controller.config.vel_limit = 3 #Original: 10

In [27]:
if PERFORM_TESTS: omc._configure_controller()

In [28]:
# @patch
# def _configure_pid(self:OdriveMotorController):
    #self.selected_axis.controller.config.pos_gain = 4.5 #Original: 1
    #self.selected_axis.controller.config.vel_gain = 3. #Original: 1.67
    #self.selected_axis.controller.config.vel_gain = 0.02 * self.odrv_axis.motor.config.torque_constant * self.odrv_axis.encoder.config.cpr #Original: 0.02 * self.odrv_axis.motor.config.torque_constant * self.odrv_axis.encoder.config.cpr
    #self.selected_axis.controller.config.vel_integrator_gain = 2.2 #Default: 0.333
    #self.selected_axis.controller.config.vel_integrator_gain = 0.1 * self.odrv_axis.motor.config.torque_constant * self.odrv_axis.encoder.config.cpr #Original: 0.1 * self.odrv_axis.motor.config.torque_constant * self.odrv_axis.encoder.config.cpr
        
        

In [29]:

omc.selected_axis.controller.config.control_mode = CONTROL_MODE_VELOCITY_CONTROL
omc.selected_axis.controller.config.input_mode = INPUT_MODE_VEL_RAMP
omc.selected_axis.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL


In [30]:
omc.odrv.error, omc.odrv.axis0.error, omc.odrv.axis0.motor.error, omc.odrv.axis0.encoder.error

(0, 0, 0, 0)

In [31]:
omc.odrv.clear_errors()

In [33]:
omc.selected_axis.controller.config.vel_integrator_gain = 2
omc.selected_axis.controller.config.vel_gain = 6. #.167
# omc.selected_axis.controller.config.vel_integrator_limit = 1

In [34]:
###TUNE VELOCITY GAIN.  Vibrations at 13, so setting to 6.
# try:
#     omc.selected_axis.controller.config.vel_gain = 6. #.167
#     for vg_step in range(10):
#         assert (omc.odrv.error, omc.odrv.axis0.error, omc.odrv.axis0.motor.error, omc.odrv.axis0.encoder.error) == (0,0,0,0)
#         for i in range(1,4):
#             omc.selected_axis.controller.input_vel = i
#             time.sleep(.5)
#         print(omc.selected_axis.controller.vel_setpoint)
#         print(omc.selected_axis.encoder.vel_estimate)
#         for i in range(50):
#             time.sleep(.1)
#             print(omc.selected_axis.encoder.vel_estimate)
#         for i in range(20):
#             time.sleep(.1)
#             omc.selected_axis.controller.input_vel = omc.selected_axis.controller.input_vel * .8
#         omc.selected_axis.controller.input_vel = 0
#         omc.selected_axis.controller.config.vel_gain = omc.selected_axis.controller.config.vel_gain * 1.3
#         print('vel_gain',omc.selected_axis.controller.config.vel_gain)
#         time.sleep(1)
# except:
#     omc.selected_axis.requested_state = AXIS_STATE_IDLE
#     print('idle successful')
#     omc.selected_axis.controller.input_vel = 0
#     print(omc.selected_axis.controller.config.vel_integrator_gain)
    

In [35]:
omc.selected_axis.controller.input_vel = .0

In [36]:
# if PERFORM_TESTS: omc._configure_pid()

In [38]:
(
    omc.selected_axis.controller.config.vel_limit,
    omc.selected_axis.controller.config.vel_limit_tolerance,
    omc.selected_axis.controller.config.vel_ramp_rate,
    omc.selected_axis.controller.config.vel_gain,
    # omc.selected_axis.controller.config.vel_integrator_limit,
    omc.selected_axis.controller.config.vel_integrator_gain,
)

(3.0, 1.2000000476837158, 1.0, 6.0, 2.0)

### Tune Position Control Gain

In [39]:
omc.selected_axis.controller.config.pos_gain = 1.2 #20.
omc.selected_axis.controller.config.input_filter_bandwidth = 10.0

In [40]:
omc.selected_axis.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
omc.selected_axis.controller.config.input_mode = INPUT_MODE_POS_FILTER
omc.selected_axis.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL

In [41]:
# try:
#     print('starting test')
#     for i in range(2):
#         omc.selected_axis.controller.input_pos = (i % 2) * 2
#         time.sleep(5)
# except:
#     omc.selected_axis.requested_state = AXIS_STATE_IDLE
#     print('idle successful')
#     omc.selected_axis.controller.input_pos = 0

In [42]:
omc.odrv.error, omc.odrv.axis0.error, omc.odrv.axis0.motor.error, omc.odrv.axis0.encoder.error

(0, 0, 0, 0)

In [43]:
omc.selected_axis.requested_state = AXIS_STATE_IDLE

In [44]:
omc._save_config()

05:10:52.628549627 [LEGACY_OBJ] protocol failed with 3 - propagating error to application


Configuration Saved.


In [45]:
omc._reboot_odrive()

Rebooting Odrive...


05:11:04.617861408 [LEGACY_OBJ] protocol failed with 3 - propagating error to application


In [46]:
omc.odrv.error, omc.odrv.axis0.error, omc.odrv.axis0.motor.error, omc.odrv.axis0.encoder.error

(0, 0, 0, 0)